### 导入数据

In [1]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
from tensorflow.contrib.slim.nets import vgg

IMAGE_SIZE = vgg.vgg_16.default_image_size


import os
import numpy as np
import cv2
import time

def input_data(npz_file):
    if os.path.exists(npz_file) :
        bird_data = np.load(npz_file)
        return bird_data['train_img'],bird_data['test_img'],bird_data['train_label'],bird_data['test_label']
    else:      
        data_path = os.path.join('../../..','data','CUB_200_2011')
        print(os.listdir(data_path))

        train_test_split_file = os.path.join(data_path,'train_test_split.txt')
        with open(train_test_split_file,'r') as file:
            train_test_split = np.array([i.split()[1] for i in file.readlines()]).astype('bool')
        print(train_test_split,train_test_split.size)

        img_paths_file = os.path.join(data_path,'images.txt')
        with open(img_paths_file,'r') as file:
            img_paths = [i.split()[1] for i in file.readlines()]
        print(img_paths[:1],len(img_paths))

        img_labels_file = os.path.join(data_path,'image_class_labels.txt')
        with open(img_labels_file,'r') as file:
            img_labels = np.array([i.split()[1] for i in file.readlines()]).astype('int')
        print(img_labels,len(img_labels))

        img_dir = os.path.join(data_path,'images')

        img_paths_train = [os.path.join(img_dir,os.path.sep.join(path.split('/'))) for i,path in enumerate(img_paths) if train_test_split[i]]
        print(img_paths_train[:1],len(img_paths_train))
        img_paths_test = [os.path.join(img_dir,os.path.sep.join(path.split('/'))) for i,path in enumerate(img_paths) if not train_test_split[i]]
        print(img_paths_test[:1],len(img_paths_test))

        train_img = np.array([cv2.resize(cv2.imread(i),(224,224)) for i in img_paths_train])
        test_img = np.array([cv2.resize(cv2.imread(i),(224,224)) for i in img_paths_test])
        train_label = np.array([l for i,l in enumerate(img_labels) if train_test_split[i] ])
        test_label = np.array([l for i,l in enumerate(img_labels) if not train_test_split[i]])
        print(train_label,train_label.size)
        print(test_label,test_label.size)

        np.savez(npz_file,train_img=train_img,test_img=test_img,train_label=train_label,test_label=test_label)
        return train_img,test_img,train_label,test_label
    
x_train,x_test,y_train,y_test = input_data('bird_data_224.npz')


num_classes = 200

# 数据预处理，把 0-255的灰度值转成 0-1 之间的浮点数
x_train = x_train.astype('float32')/255
x_test = x_test.astype('float32')/255
y_train, y_test = np.array(y_train)-1, np.array(y_test)-1



print('type:',type(x_train),type(y_train))
print('shape:',x_train.shape,y_train.shape)
print('size:',x_train.size,y_train.size)

type: <class 'numpy.ndarray'> <class 'numpy.ndarray'>
shape: (5994, 224, 224, 3) (5994,)
size: 902264832 5994


### 批量数据 生成器

In [2]:
# 遍历一遍数据集，不打乱
def batch_generator(data, batch_size):
    batch_count = 0 
    while batch_count * batch_size < len(data[0]):
        start = batch_count * batch_size
        end = np.min([start + batch_size,len(data[0])])
        batch_count += 1
        yield [d[start:end] for d in data]

# 测试
for batch in batch_generator([[1,2,3,4,5,6,7,8,9]],4):
    print(batch)

[[1, 2, 3, 4]]
[[5, 6, 7, 8]]
[[9]]


### 计算数据集特征

In [3]:
batch_size = 128


#用于保存微调后的检查点文件和日志文件路径
train_log_dir = './vgg_16_2016_08_28/slim_fine_tune'
train_log_file = 'birds_fine_tune.ckpt'


assert(tf.gfile.Exists(train_log_dir) == True)

#创建一个图，作为当前图
with tf.Graph().as_default():

    #加载数据
    image_holder = tf.placeholder(tf.float32, [None, IMAGE_SIZE, IMAGE_SIZE, 3])
    label_holder = tf.placeholder(tf.int32, [None,]) 
    is_training = tf.placeholder(dtype = tf.bool)

    #创建vgg16网络  如果想冻结所有层，可以指定slim.conv2d中的 trainable=False
    logits,end_points =  vgg.vgg_16(image_holder, is_training=is_training, num_classes = num_classes)  
    
    for i in end_points:
        print(i)


    #预测标签
    pred = tf.cast(tf.argmax(logits,axis=1),tf.int32)

    #预测结果评估        
    correct = tf.equal(pred, label_holder)                    #返回一个数组 表示统计预测正确或者错误 
    accuracy = tf.reduce_mean(tf.cast(correct,tf.float32))                #求准确率

    num_batch = x_test.shape[0] // batch_size

    #用于保存检查点文件 
    save = tf.train.Saver(max_to_keep=1) 

    #恢复模型
    with tf.Session() as sess:      
        sess.run(tf.global_variables_initializer())

        #检查最近的检查点文件
        ckpt = tf.train.latest_checkpoint(train_log_dir)
        if ckpt != None:
            save.restore(sess,ckpt)
            print('加载上次训练保存后的模型！')
        else:
            assert(False)
                

        total_correct = 0.0 
        feature_vecs_test = []
        for image_batch, label_batch in batch_generator([x_test,y_test], batch_size):                                                            
            accuracy_value,layer_outputs = sess.run([accuracy, end_points],feed_dict = {image_holder:image_batch,label_holder:label_batch,is_training:False})
            correct_num = image_batch.shape[0]*accuracy_value
            print('batch_size: {:2d} acc: {:.3f} correct_num: {:.1f}'.format(image_batch.shape[0],accuracy_value,correct_num))
            total_correct += correct_num
            
            feature = layer_outputs['vgg_16/fc7'].squeeze()
            feature_vecs_test.append(feature)
        feature_vecs_test = np.vstack(feature_vecs_test)

        print('测试集准确率: {:.3f}'.format(total_correct/x_test.shape[0]))
        
        total_correct = 0.0 
        feature_vecs_train = []
        for image_batch, label_batch in batch_generator([x_train,y_train], batch_size):                                                            
            accuracy_value,layer_outputs = sess.run([accuracy, end_points],feed_dict = {image_holder:image_batch,label_holder:label_batch,is_training:False})
            correct_num = image_batch.shape[0]*accuracy_value
            print('batch_size: {:2d} acc: {:.3f} correct_num: {:.1f}'.format(image_batch.shape[0],accuracy_value,correct_num))
            total_correct += correct_num
            
            feature = layer_outputs['vgg_16/fc7'].squeeze()
            feature_vecs_train.append(feature)
        feature_vecs_train = np.vstack(feature_vecs_train)

        print('训练集准确率: {:.3f}'.format(total_correct/x_train.shape[0]))
        

print(feature_vecs_train.shape,feature_vecs_test.shape)

def normalize(a):
    return np.diag(1/np.sqrt(np.sum(np.square(a),axis=1))).dot(a) #归一化

feature_vecs_train, feature_vecs_test = normalize(feature_vecs_train),normalize(feature_vecs_test)

np.savez('vgg_feature_vecs',train=feature_vecs_train,test=feature_vecs_test)

vgg_16/conv1/conv1_1
vgg_16/conv1/conv1_2
vgg_16/pool1
vgg_16/conv2/conv2_1
vgg_16/conv2/conv2_2
vgg_16/pool2
vgg_16/conv3/conv3_1
vgg_16/conv3/conv3_2
vgg_16/conv3/conv3_3
vgg_16/pool3
vgg_16/conv4/conv4_1
vgg_16/conv4/conv4_2
vgg_16/conv4/conv4_3
vgg_16/pool4
vgg_16/conv5/conv5_1
vgg_16/conv5/conv5_2
vgg_16/conv5/conv5_3
vgg_16/pool5
vgg_16/fc6
vgg_16/fc7
vgg_16/fc8
INFO:tensorflow:Restoring parameters from ./vgg_16_2016_08_28/slim_fine_tune\birds_fine_tune.ckpt-20
加载上次训练保存后的模型！
batch_size: 128 acc: 0.453 correct_num: 58.0
batch_size: 128 acc: 0.711 correct_num: 91.0
batch_size: 128 acc: 0.602 correct_num: 77.0
batch_size: 128 acc: 0.688 correct_num: 88.0
batch_size: 128 acc: 0.562 correct_num: 72.0
batch_size: 128 acc: 0.414 correct_num: 53.0
batch_size: 128 acc: 0.383 correct_num: 49.0
batch_size: 128 acc: 0.594 correct_num: 76.0
batch_size: 128 acc: 0.484 correct_num: 62.0
batch_size: 128 acc: 0.586 correct_num: 75.0
batch_size: 128 acc: 0.547 correct_num: 70.0
batch_size: 128 acc

### 计算 mAP

In [4]:

# 将训练集作为 检索库
x_database,y_database = x_train, y_train
feature_vecs_database = feature_vecs_train


# 从 v_set 中找出和 v 最相似的 k 个元素
def topK(v, v_set, k):
    dist = np.array([distance(v,i) for i in v_set])
    idx = np.argpartition(dist, k)[:k]
#   return dist.argsort()[:k][::-1]
    return idx[np.argsort(dist[idx])]

def distance(a,b):
    return -a.dot(b) # 余弦距离
    return np.sum(np.square(a-b)) # 欧式距离

def onehot2class(onehot):
    return np.argmax(onehot) #0-199

def retrieve(x,k):
    featone = feature_model.predict(np.expand_dims(x, axis=0))
    return topK(featone[0],feature_vecs_database,k)

def AP(idxK,y):
    bool_list = np.array([yi for yi in y_database[idxK]]) == y
#     M = num_per_class[y]
    M = np.sum(bool_list)
    if M == 0:
        return 0
    return np.sum(np.add.accumulate(bool_list)*bool_list.astype(int)/
                  (np.array([i+1 for i in range(len(bool_list))])))/M

def mAP(k):
    feature_vecs = feature_vecs_test
    idxKs = [topK(featone,feature_vecs_database,k) for featone in feature_vecs]
    return np.average([ AP(idxK,y) for idxK,y in zip(idxKs,y_test)])

import time

for k in [1,5,10,50]:
    beg = time.time()
    print("mAP @ {}: {:.3f}".format(k,mAP(k)))
    end = time.time()
    print('time used: {:.3f}s'.format(end-beg))

# print(feature_model.predict(np.expand_dims(x_test[32], axis=0))) # 稀疏的特征

mAP @ 1: 0.580
time used: 111.193s
mAP @ 5: 0.629
time used: 113.267s
mAP @ 10: 0.613
time used: 117.158s
mAP @ 50: 0.525
time used: 121.564s
